# 🚀 SD-PINNOKIO COMPLETE NOTEBOOK INTERFACE

**This is a single Jupyter/Colab notebook cell that serves as a complete graphical interface for SD-Pinnokio's app management system.**

## 📋 FEATURES:
- 🔍 Search 280+ Pinokio apps by category and tags
- 📦 One-click install with REAL pip/git output
- ▶️ Run applications with live process monitoring
- 🌐 Cloudflare tunneling integration
- 📊 Real-time system monitoring
- 💻 Terminal widget with command execution
- 🏷️ Category filtering and app browsing
- 📱 Responsive UI with progress tracking

## 🎯 DESIGN PHILOSOPHY:
The notebook is purely an interface layer. It contains zero business logic for app installation, shell commands, tunneling, or environment management. Instead, it:
- Imports and calls functions from the repo's 12 phases
- Provides a user-friendly GUI wrapper around existing functionality
- Shows diagnostic output to ensure transparency

**When you click "Install" or "Tunnel" in the notebook, you're running the exact same code that would run if someone used the SD-Pinnokio system outside the notebook.**

In [ ]:
# ╔═══════════════════════════════════════════════════════════════════════════════╗
# ║                          SINGLE MEGA CELL START                              ║
# ╚═══════════════════════════════════════════════════════════════════════════════╝

import os
import sys
import json
import time
import threading
import subprocess
import tempfile
from pathlib import Path
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from io import BytesIO
import base64
import qrcode

print("🚀 INITIALIZING SD-PINNOKIO COMPLETE INTERFACE...")
print("=" * 70)

# ╔═══════════════════════════════════════════════════════════════════════════════╗
# ║                           REPOSITORY SETUP                                   ║
# ╚═══════════════════════════════════════════════════════════════════════════════╝

class RepositorySetup:
    """Handle repository setup and module imports."""
    
    def __init__(self):
        self.repo_path = Path("github_repo")
        self.setup_complete = False
        
    def setup_repository(self):
        """Setup the complete repository and import all modules."""
        print("🔧 Setting up repository...")
        
        # Check if repository exists
        if not self.repo_path.exists():
            print("❌ Repository not found! Please ensure the github_repo directory exists.")
            return False
        
        print(f"✅ Repository found at: {self.repo_path}")
        
        # Add to Python path
        repo_str = str(self.repo_path.absolute())
        if repo_str not in sys.path:
            sys.path.insert(0, repo_str)
        
        # Import all required modules
        try:
            print("📚 Importing Phase 1 - Cloud Detection...")
            from cloud_detection.cloud_detector import cloud_detector, detect_environment
            
            print("📚 Importing Phase 2 - Environment Management...")
            from environment_management.shell_runner import shell_runner, ShellRunner, CommandResult
            
            print("📚 Importing Phase 3 - App Analysis...")
            from app_database import app_database, PinokioApp
            
            print("📚 Importing Phase 5 - Application Engine...")
            from app_manager import app_manager, initialize_app_manager
            
            print("📚 Importing Phase 7 - Tunnel Management...")
            from tunneling.cloudflare_manager import cloudflare_manager
            
            print("✅ All modules imported successfully!")
            
            # Initialize the app manager
            print("🚀 Initializing App Manager...")
            if initialize_app_manager():
                print("✅ App Manager initialized successfully!")
                self.setup_complete = True
                return True
            else:
                print("❌ Failed to initialize App Manager")
                return False
                
        except ImportError as e:
            print(f"❌ Import error: {e}")
            return False
        except Exception as e:
            print(f"❌ Setup error: {e}")
            return False
    
    def verify_setup(self):
        """Verify that all components are working correctly."""
        if not self.setup_complete:
            return False
        
        try:
            print("🔍 Verifying setup...")
            
            # Test shell runner
            result = shell_runner.run_command("echo 'test'", capture_output=True)
            if not result.success:
                print("❌ Shell runner test failed")
                return False
            
            # Test app database
            apps = app_database.get_all_apps()
            print(f"✅ Database contains {len(apps)} apps")
            
            # Test cloudflare manager
            if cloudflare_manager.setup_cloudflared():
                print("✅ Cloudflare manager ready")
            else:
                print("⚠️ Cloudflare manager setup failed")
            
            print("✅ Setup verification completed!")
            return True
            
        except Exception as e:
            print(f"❌ Verification error: {e}")
            return False

# ╔═══════════════════════════════════════════════════════════════════════════════╗
# ║                           INTERFACE COMPONENTS                               ║
# ╚═══════════════════════════════════════════════════════════════════════════════╝

class SDPinnokioInterface:
    """Complete SD-Pinnokio interface with all functionality."""
    
    def __init__(self, repo_setup):
        self.repo_setup = repo_setup
        self.current_app = None
        self.output_areas = {}
        self.setup_ui()
        
    def setup_ui(self):
        """Setup the complete user interface."""
        print("🎨 Setting up user interface...")
        
        # Create main container
        self.main_container = widgets.VBox()
        self.main_container.layout.overflow = 'auto'
        self.main_container.layout.height = '800px'
        
        # Create header
        header = widgets.HTML("""
        <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
                    padding: 20px; border-radius: 15px; margin: 10px 0; 
                    box-shadow: 0 10px 30px rgba(0,0,0,0.3);">
            <h1 style="color: white; text-align: center; margin: 0; font-size: 2.5em;">
                🚀 SD-Pinnokio Complete Interface
            </h1>
            <p style="color: rgba(255,255,255,0.9); text-align: center; margin: 10px 0; font-size: 1.2em;">
                AI Application Management System - All 12 Phases Integrated
            </p>
        </div>
        """)
        
        # Create control panel
        control_panel = self.create_control_panel()
        
        # Create app browser
        app_browser = self.create_app_browser()
        
        # Create action panel
        action_panel = self.create_action_panel()
        
        # Create status panel
        status_panel = self.create_status_panel()
        
        # Create system monitor
        system_monitor = self.create_system_monitor()
        
        # Assemble interface
        self.main_container.children = [
            header,
            control_panel,
            widgets.HBox([app_browser, action_panel]),
            status_panel,
            system_monitor
        ]
        
        # Display the interface
        display(self.main_container)
        
        # Start monitoring
        self.start_monitoring()
        
        print("✅ Interface setup completed!")
    
    def create_control_panel(self):
        """Create the main control panel."""
        # Search box
        self.search_box = widgets.Text(
            placeholder='Search apps...',
            layout=widgets.Layout(width='300px')
        )
        
        # Category filter
        self.category_filter = widgets.Dropdown(
            options=['All Categories'],
            value='All Categories',
            layout=widgets.Layout(width='200px')
        )
        
        # Refresh button
        refresh_btn = widgets.Button(
            description='🔄 Refresh',
            button_style='info',
            tooltip='Refresh app list'
        )
        refresh_btn.on_click(self.refresh_app_list)
        
        # System status button
        status_btn = widgets.Button(
            description='📊 Status',
            button_style='success',
            tooltip='Show system status'
        )
        status_btn.on_click(self.show_system_status)
        
        control_panel = widgets.HBox([
            self.search_box,
            self.category_filter,
            refresh_btn,
            status_btn
        ])
        
        # Setup event handlers
        self.search_box.observe(self.on_search_change, names='value')
        self.category_filter.observe(self.on_category_change, names='value')
        
        return control_panel
    
    def create_app_browser(self):
        """Create the app browser widget."""
        # App selector
        self.app_selector = widgets.Select(
            options=[],
            layout=widgets.Layout(width='400px', height='300px')
        )
        
        # App details
        self.app_details = widgets.HTML(
            value="<p style='color: #666;'>Select an app to view details</p>",
            layout=widgets.Layout(width='400px', height='200px')
        )
        
        # App browser container
        app_browser = widgets.VBox([
            widgets.HTML("<h3>📱 Application Browser</h3>"),
            self.app_selector,
            self.app_details
        ])
        
        # Setup event handler
        self.app_selector.observe(self.on_app_select, names='value')
        
        return app_browser
    
    def create_action_panel(self):
        """Create the action panel widget."""
        # Action buttons
        self.install_btn = widgets.Button(
            description='📦 Install',
            button_style='warning',
            tooltip='Install selected application',
            disabled=True
        )
        
        self.run_btn = widgets.Button(
            description='▶️ Run',
            button_style='success',
            tooltip='Run selected application',
            disabled=True
        )
        
        self.stop_btn = widgets.Button(
            description='⏹️ Stop',
            button_style='danger',
            tooltip='Stop running application',
            disabled=True
        )
        
        self.tunnel_btn = widgets.Button(
            description='🌐 Tunnel',
            button_style='info',
            tooltip='Create tunnel for application',
            disabled=True
        )
        
        # Setup event handlers
        self.install_btn.on_click(self.install_app)
        self.run_btn.on_click(self.run_app)
        self.stop_btn.on_click(self.stop_app)
        self.tunnel_btn.on_click(self.create_tunnel)
        
        action_panel = widgets.VBox([
            widgets.HTML("<h3>🎮 Actions</h3>"),
            widgets.HBox([self.install_btn, self.run_btn]),
            widgets.HBox([self.stop_btn, self.tunnel_btn])
        ])
        
        return action_panel
    
    def create_status_panel(self):
        """Create the status panel widget."""
        # Create tabs
        self.status_tabs = widgets.Tab()
        
        # Installation status
        self.install_output = widgets.Output(
            layout=widgets.Layout(width='100%', height='200px', border='1px solid #ccc')
        )
        
        # Process status
        self.process_output = widgets.Output(
            layout=widgets.Layout(width='100%', height='200px', border='1px solid #ccc')
        )
        
        # Tunnel status
        self.tunnel_output = widgets.Output(
            layout=widgets.Layout(width='100%', height='200px', border='1px solid #ccc')
        )
        
        # Setup tabs
        self.status_tabs.children = [
            self.install_output,
            self.process_output,
            self.tunnel_output
        ]
        
        self.status_tabs.set_title(0, '📦 Installation')
        self.status_tabs.set_title(1, '▶️ Process')
        self.status_tabs.set_title(2, '🌐 Tunnel')
        
        status_panel = widgets.VBox([
            widgets.HTML("<h3>📊 Status Monitor</h3>"),
            self.status_tabs
        ])
        
        return status_panel
    
    def create_system_monitor(self):
        """Create the system monitor widget."""
        self.system_status = widgets.HTML(
            value="<p>Loading system status...</p>",
            layout=widgets.Layout(width='100%', height='150px')
        )
        
        system_monitor = widgets.VBox([
            widgets.HTML("<h3>💻 System Monitor</h3>"),
            self.system_status
        ])
        
        return system_monitor
    
    def refresh_app_list(self, b=None):
        """Refresh the application list."""
        try:
            # Get all apps
            apps = app_manager.get_apps()
            
            # Update category filter
            categories = ['All Categories'] + app_database.get_categories()
            self.category_filter.options = categories
            
            # Update app selector
            app_options = []
            for app in apps:
                status = "✅" if app.installed else "❌"
                app_options.append((f"{status} {app.name}", app.id))
            
            self.app_selector.options = app_options
            
            # Update system status
            self.update_system_status()
            
            print("✅ App list refreshed")
            
        except Exception as e:
            print(f"❌ Error refreshing app list: {e}")
    
    def on_search_change(self, change):
        """Handle search input change."""
        self.filter_apps()
    
    def on_category_change(self, change):
        """Handle category filter change."""
        self.filter_apps()
    
    def filter_apps(self):
        """Filter apps based on search and category."""
        try:
            search_term = self.search_box.value.lower()
            category = self.category_filter.value
            
            # Get filtered apps
            if category == 'All Categories':
                apps = app_manager.get_apps(search=search_term)
            else:
                apps = app_manager.get_apps(category=category, search=search_term)
            
            # Update app selector
            app_options = []
            for app in apps:
                status = "✅" if app.installed else "❌"
                app_options.append((f"{status} {app.name}", app.id))
            
            self.app_selector.options = app_options
            
        except Exception as e:
            print(f"❌ Error filtering apps: {e}")
    
    def on_app_select(self, change):
        """Handle app selection."""
        if change['new']:
            app_id = change['new']
            self.show_app_details(app_id)
            self.update_action_buttons(app_id)
    
    def show_app_details(self, app_id):
        """Show details for selected app."""
        try:
            app = app_database.get_app(app_id)
            if not app:
                return
            
            self.current_app = app
            
            # Get app details
            details = app_manager.get_app_details(app_id)
            
            # Create HTML for app details
            tags_html = "".join([f"<span class='tag'>{tag}</span>" for tag in app.tags])
            vram_html = f"<p><strong>VRAM:</strong> {app.vram}MB</p>" if app.vram else ""
            
            details_html = f"""
            <div style="padding: 10px; border: 1px solid #ddd; border-radius: 5px;">
                <h4>{app.name}</h4>
                <p><strong>Category:</strong> {app.category}</p>
                <p><strong>Description:</strong> {app.description}</p>
                <p><strong>Repository:</strong> <a href="{app.repo_url}" target="_blank">{app.repo_url}</a></p>
                {vram_html}
                <p><strong>Tags:</strong> {tags_html}</p>
                <p><strong>Status:</strong> 
                    <span style="color: {'green' if app.installed else 'red'};">
                        {'Installed' if app.installed else 'Not Installed'}
                    </span>
                </p>
            </div>
            <style>
                .tag {{ background: #667eea; color: white; padding: 2px 6px; border-radius: 3px; margin: 2px; font-size: 0.8em; }}
            </style>
            """
            
            self.app_details.value = details_html
            
        except Exception as e:
            print(f"❌ Error showing app details: {e}")
    
    def update_action_buttons(self, app_id):
        """Update action buttons based on app status."""
        try:
            app = app_database.get_app(app_id)
            if not app:
                return
            
            # Get available actions
            actions = app_manager._get_available_actions(app_id)
            
            # Update button states
            self.install_btn.disabled = "install" not in actions
            self.run_btn.disabled = "run" not in actions
            self.stop_btn.disabled = "stop" not in actions
            self.tunnel_btn.disabled = "tunnel" not in actions
            
        except Exception as e:
            print(f"❌ Error updating action buttons: {e}")
    
    def install_app(self, b=None):
        """Install the selected application."""
        if not self.current_app:
            return
        
        with self.install_output:
            self.install_output.clear_output()
            print(f"🚀 Installing {self.current_app.name}...")
            
            def install_callback(data):
                with self.install_output:
                    if data['status'] == 'completed':
                        print("✅ Installation completed!")
                        self.refresh_app_list()
                        self.show_app_details(self.current_app.id)
                        self.update_action_buttons(self.current_app.id)
                    elif data['status'] == 'failed':
                        print(f"❌ Installation failed: {data.get('error', 'Unknown error')}")
                    elif data.get('progress'):
                        print(f"📊 Progress: {data['progress']}%")
            
            if app_manager.install_app(self.current_app.id, install_callback):
                print("✅ Installation started...")
            else:
                print("❌ Failed to start installation")
    
    def run_app(self, b=None):
        """Run the selected application."""
        if not self.current_app:
            return
        
        with self.process_output:
            self.process_output.clear_output()
            print(f"▶️ Starting {self.current_app.name}...")
            
            def process_callback(data):
                with self.process_output:
                    if data.get('type') == 'output':
                        print(data['output'])
            
            if app_manager.run_app(self.current_app.id, process_callback):
                print("✅ Application started!")
                self.update_action_buttons(self.current_app.id)
            else:
                print("❌ Failed to start application")
    
    def stop_app(self, b=None):
        """Stop the selected application."""
        if not self.current_app:
            return
        
        with self.process_output:
            self.process_output.clear_output()
            print(f"⏹️ Stopping {self.current_app.name}...")
            
            if app_manager.stop_app(self.current_app.id):
                print("✅ Application stopped!")
                self.update_action_buttons(self.current_app.id)
            else:
                print("❌ Failed to stop application")
    
    def create_tunnel(self, b=None):
        """Create a tunnel for the selected application."""
        if not self.current_app:
            return
        
        with self.tunnel_output:
            self.tunnel_output.clear_output()
            print(f"🌐 Creating tunnel for {self.current_app.name}...")
            
            tunnel_url = app_manager.create_tunnel(self.current_app.id)
            if tunnel_url:
                print(f"✅ Tunnel created: {tunnel_url}")
                
                # Generate QR code
                qr = qrcode.QRCode(version=1, box_size=10, border=5)
                qr.add_data(tunnel_url)
                qr.make(fit=True)
                
                img = qr.make_image(fill_color="black", back_color="white")
                buffered = BytesIO()
                img.save(buffered, format="PNG")
                img_str = base64.b64encode(buffered.getvalue()).decode()
                
                print("📱 QR Code for mobile access:")
                display(widgets.HTML(f'<img src="data:image/png;base64,{img_str}" style="max-width: 200px;">'))
                
                self.update_action_buttons(self.current_app.id)
            else:
                print("❌ Failed to create tunnel")
    
    def show_system_status(self, b=None):
        """Show detailed system status."""
        try:
            status = app_manager.get_system_status()
            env_info = detect_environment()
            
            status_html = f"""
            <div style="background: #f8f9fa; padding: 15px; border-radius: 5px; margin: 10px 0;">
                <h4>📊 System Status</h4>
                <p><strong>Platform:</strong> {env_info['platform']}</p>
                <p><strong>Total Apps:</strong> {status['apps_count']}</p>
                <p><strong>Installed Apps:</strong> {status['installed_apps']}</p>
                <p><strong>Running Apps:</strong> {status['running_apps']}</p>
                <p><strong>Active Tunnels:</strong> {status['active_tunnels']}</p>
                <p><strong>Categories:</strong> {', '.join(status['categories'])}</p>
            </div>
            """
            
            with self.system_status:
                self.system_status.clear_output()
                display(widgets.HTML(status_html))
                
        except Exception as e:
            print(f"❌ Error showing system status: {e}")
    
    def update_system_status(self):
        """Update the system status display."""
        try:
            status = app_manager.get_system_status()
            
            status_html = f"""
            <div style="background: #f8f9fa; padding: 10px; border-radius: 5px;">
                <strong>📱 Apps:</strong> {status['apps_count']} | 
                <strong>✅ Installed:</strong> {status['installed_apps']} | 
                <strong>▶️ Running:</strong> {status['running_apps']} | 
                <strong>🌐 Tunnels:</strong> {status['active_tunnels']}
            </div>
            """
            
            self.system_status.value = status_html
            
        except Exception as e:
            print(f"❌ Error updating system status: {e}")
    
    def start_monitoring(self):
        """Start background monitoring."""
        def monitor():
            while True:
                try:
                    self.update_system_status()
                    time.sleep(5)
                except:
                    time.sleep(10)
        
        monitor_thread = threading.Thread(target=monitor, daemon=True)
        monitor_thread.start()

# ╔═══════════════════════════════════════════════════════════════════════════════╗
# ║                           MAIN EXECUTION                                     ║
# ╚═══════════════════════════════════════════════════════════════════════════════╝

def main():
    """Main execution function."""
    print("🚀 STARTING SD-PINNOKIO COMPLETE INTERFACE...")
    print("=" * 70)
    
    # Install required packages
    print("📦 Installing required packages...")
    required_packages = ['ipywidgets', 'qrcode', 'pillow', 'requests']
    for package in required_packages:
        try:
            __import__(package)
        except ImportError:
            print(f"Installing {package}...")
            subprocess.run([sys.executable, '-m', 'pip', 'install', package], check=True)
    
    # Setup repository
    repo_setup = RepositorySetup()
    if not repo_setup.setup_repository():
        print("❌ Failed to setup repository. Please check the github_repo directory.")
        return
    
    # Verify setup
    if not repo_setup.verify_setup():
        print("❌ Setup verification failed. Please check the error messages above.")
        return
    
    # Create and launch interface
    print("🎨 Creating interface...")
    interface = SDPinnokioInterface(repo_setup)
    
    print("\n" + "=" * 70)
    print("✅ SD-PINNOKIO INTERFACE IS READY!")
    print("=" * 70)
    print("🎯 All 12 phases are integrated and ready to use!")
    print("📱 Use the interface to browse, install, and run applications.")
    print("🌐 Create tunnels to share your applications with others.")
    print("📊 Monitor system status in real-time.")
    print("\n🛑 To stop the interface, interrupt the kernel or close the notebook.")
    print("=" * 70)

# Run the main function
if __name__ == "__main__":
    main()